In [1]:
import pandas as pd

import numpy as np

import networkx as nx

import matplotlib.pyplot as plt

from matplotlib import font_manager as fm, rcParams

from itertools import combinations

plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签
plt.rcParams['axes.unicode_minus']=False   #这两行需要手动设置

pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 100)

In [6]:
df = pd.read_csv('table.csv', encoding = "GBK", header=1)

df.head()

,code,year,month,name,box2019,box,director1,director2,dfans,dst,ms1,ms1fans,ms1st,fs1,fs1fans,fs1st,ms2,ms2fans,ms2st,fs2,fs2fans,fs2st,guests,gsfans,gst,type,love,crime,history,other,family,animated,comedy,action,jjuj,jjij,hbij,jxiaj,jmaj,jluj,bhuj,jxioj,rank,total,sequels,remake,prize
0,1,2009,9,建国大业,5.369108,4.16,韩三平,黄建新,5456.0,2011/3/23,唐国强,37179.0,2011/3/12,许晴,8381242.0,2011/3/23,张国立,475478.0,2011/3/13,邬君梅,403431.0,2010/3/2,陈坤,78031895.0,2009/11/13,历史、战争,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,6.1,111541.0,0.0,1.0,2011华表奖优秀故事片奖、2010金鹿奖最佳华语故事片、2010百花奖最佳影片奖
1,2,2009,12,十月围城,3.755794,2.91,陈德森,NaN,545022.0,2010/3/9,甄子丹,6078838.0,2010/3/30,范冰冰,39431925.0,2013/12/25,梁家辉,0.0,NaN,李宇春,3517480.0,2009/9/10,谢霆锋,NaN,NaN,历史、战争,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,7.7,204435.0,0.0,1.0,2011华表奖优秀合拍片奖、2010金像奖最佳电影第47届台湾电影金马奖
2,3,2009,1,赤壁(下),3.368599,2.61,吴宇森,NaN,233738.0,2015/7/16,梁朝伟,1504853.0,2010/5/27,赵薇,73974278.0,2009/10/28,金城武,0.0,NaN,林志玲,9901922.0,2009/8/28,佟大为,22612580.0,2009/8/28,历史、战争,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,6.5,84042.0,1.0,1.0,2009华表奖优秀合拍片奖，香港金像奖最佳剪接
3,4,2009,12,三枪拍案惊奇,3.329879,2.58,张艺谋,NaN,0.0,NaN,孙红雷,11381520.0,2009/9/29,闫妮,377610.0,2011/6/28,小沈阳,12835247.0,2009/8/28,NaN,NaN,NaN,赵本山,5524638.0,2010/11/20,惊悚、悬疑、犯罪,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.7,68824.0,0.0,0.0,NaN
4,5,2009,9,风声,2.891058,2.24,高群书,陈国富,0.0,NaN,黄晓明,39327844.0,2010/4/14,周迅,4309367.0,2011/5/4,苏有朋,22655876.0,2010/12/14,李冰冰,35118659.0,2009/8/28,张涵予,636765.0,2009/8/28,惊悚、悬疑、犯罪,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,8.0,234005.0,0.0,1.0,2011华表奖优秀合拍片奖第30届百花奖第46届金马奖


In [7]:
attr_list = ['name','box2019','director1','director2','ms1', 'fs1','ms2','fs2','guests']

df = df[attr_list]

In [8]:
df.loc[:, ['name', 'director1', 'director2', 'ms1', 'fs1', 'ms2', 'fs2',
       'guests']] = df[['name', 'director1', 'director2', 'ms1', 'fs1', 'ms2', 'fs2',
       'guests']].astype('str')

In [9]:
for attr in attr_list:
    
    if attr != 'box2019':
        
        df.loc[:, attr] = df[attr].apply(lambda x: x.replace(' ',',').lstrip(' ').rstrip(' '))

df.loc[:,'director1'] = df['director1'].replace('nan', pd.NA)

df = df[~df['director1'].isna()].reset_index(drop=True)

In [10]:
df.loc[:, 'dir'] = df['director1'] + ',' + df['director2']

In [11]:
df['dict'] = [(a, b.split(',')) for a, b in zip(df['name'],df['dir'])]

In [12]:
film_dir_array = df['dict'].values

In [12]:
film_dir_array

array([('建国大业', ['韩三平', '黄建新']), ('十月围城', ['陈德森', 'nan']),
       ('赤壁(下)', ['吴宇森', 'nan']), ..., ('相亲时代', ['蒋君君', 'nan']),
       ('薪火相传', ['王鼎', 'nan']), ('大陈岛誓言', ['张笑君', 'nan'])], dtype=object)

In [2]:
import bs4

import requests

import time

In [120]:
url = f'http://baike.baidu.com/item/{search}'

response = requests.get(url, headers=headers)

if response.status_code != 200:
    print('error! fail to retrieve')
    #break

response.encoding = response.apparent_encoding

html = response.text

soup = bs4.BeautifulSoup(html, 'html.parser')

seq1 = 'None'

seq2 = 'None'

link1 = soup.find_all('dt', class_='basicInfo-item name')

In [131]:
link4

[<dd class="basicInfo-item value">
 笔仙Ⅲ
 </dd>, <dd class="basicInfo-item value">
 Bi Xian 3；Bunshinsaba Ⅲ
 </dd>, <dd class="basicInfo-item value">
 笔仙3
 </dd>, <dd class="basicInfo-item value">
 2014年
 </dd>, <dd class="basicInfo-item value">
 <a data-lemmaid="9119953" href="/item/%E4%B8%AD%E5%9B%BD%E7%94%B5%E5%BD%B1%E8%82%A1%E4%BB%BD%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8/9119953" target="_blank">中国电影股份有限公司</a>
 </dd>, <dd class="basicInfo-item value">
 中国
 </dd>, <dd class="basicInfo-item value">
 3000万
 </dd>, <dd class="basicInfo-item value">
 <a href="/item/%E5%AE%89%E5%85%B5%E5%9F%BA" target="_blank">安兵基</a>
 </dd>, <dd class="basicInfo-item value">
 <a href="/item/%E6%9D%A8%E5%93%B2" target="_blank">杨哲</a>、<a href="/item/%E6%9D%8E%E9%A1%BA%E5%AE%89" target="_blank">李顺安</a>
 </dd>, <dd class="basicInfo-item value">
 <a href="/item/%E8%A3%B4%E7%90%B3" target="_blank">裴琳</a>
 </dd>, <dd class="basicInfo-item value">
 惊悚/<a data-lemmaid="113499" href="/item/%E5%89%A7%E6%83%85/113499"

In [127]:


if len(dir_list) > 1:

    directors = [stuff.text.strip('\n') for stuff in dir_list]

    director = "".join(str(item) + ', ' for item in directors)

else:

    director = link4[seq1].find('a').text

[<dd class="basicInfo-item value">
 建国大业
 </dd>, <dd class="basicInfo-item value">
 The Founding of a Republic
 </dd>, <dd class="basicInfo-item value">
 剧情片
 </dd>, <dd class="basicInfo-item value">
 中影集团等
 </dd>, <dd class="basicInfo-item value">
 中国
 </dd>, <dd class="basicInfo-item value">
 博纳影业集团、中国电影集团公司
 </dd>, <dd class="basicInfo-item value">
 韩三平、黄建新
 </dd>, <dd class="basicInfo-item value">
 王兴东、陈宝光
 </dd>, <dd class="basicInfo-item value">
 韩三平、黄建新（总制片人）<sup class="sup--normal" data-ctrmap=":3," data-sup="3">
 [3]</sup><a class="sup-anchor" name="ref_[3]_5580889"> </a>
 </dd>, <dd class="basicInfo-item value">
 唐国强、张国立、许晴、刘劲、陈坤、王伍福
 </dd>, <dd class="basicInfo-item value">
 135 分钟
 </dd>, <dd class="basicInfo-item value">
 2009年9月16日（中国内地）
 </dd>, <dd class="basicInfo-item value">
 43000 万元
 </dd>, <dd class="basicInfo-item value">
 汉语普通话、英语等
 </dd>, <dd class="basicInfo-item value">
 彩色
 </dd>, <dd class="basicInfo-item value">
 2009年
 </dd>, <dd class="basicInfo-item valu

In [122]:
for n, v in enumerate(link1):

    info = "".join(v.text.split())

    if '导演' in info:

        seq1 = n

    if '主演' in info or '演员' in info:

        seq2 = n

link2 = soup.find_all('dd', class_='basicInfo-item value')

try: 

    dir_list = link4[seq1].find_all('a')

    if len(dir_list) > 1:

        directors = [stuff.text.strip('\n') for stuff in dir_list]

        director = "".join(str(item) + ', ' for item in directors)

    else:

        director = link4[seq1].find('a').text

except:

    director = link4[seq1].text

try:

    act_list = link4[seq2].find_all('a')

    if len(act_list) > 1:

        actors = [stuff.text.strip('\n') for stuff in act_list]

        actor = "".join(str(item) + ', ' for item in actors)


    else:

        actor = link4[seq2].find('a').text

except:

    actor = 'None'

film_dir_act[search]['dir'] = director

film_dir_act[search]['dir'] = actor

print(f"{search} is directed by {director} and casted {actor}")

17

http://baike.baidu.com/item/笔仙Ⅲ/20251009

In [119]:


if len(link1) == 0:

    meta = soup.find_all('div', class_='para')

    index = 0

    for n, v in enumerate(meta):

        if '演员' in v or '制片' in v or '导演' in v or '影视' in v:

            index = n

    code = meta[index].find('a').get('data-lemmaid', None)

    url = f'http://baike.baidu.com/item/{search}/{code}'

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print('error! fail to retrieve')
    #break

    response.encoding = response.apparent_encoding

    html = response.text

    soup1 = bs4.BeautifulSoup(html, 'html.parser')

    link3 = soup1.find_all('dt', class_='basicInfo-item name')


'建国大业'

In [16]:
try: 

    dir_list = link2[seq1].find_all('a')

    if len(dir_list) > 1:

        directors = [stuff.text.strip('\n') for stuff in dir_list]

        director = "".join(str(item) + ', ' for item in directors)

    else:

        director = link2[seq1].find('a').text

except:

    director = link2[seq1].text

try:

    act_list = link2[seq2].find_all('a')

    if len(act_list) > 1:

        actors = [stuff.text.strip('\n') for stuff in act_list]

        actor = "".join(str(item) + ', ' for item in actors)


    else:

        actor = link2[seq2].find('a').text

except:

    actor = 'None'

[<dd class="basicInfo-item value">
 建国大业
 </dd>, <dd class="basicInfo-item value">
 The Founding of a Republic
 </dd>, <dd class="basicInfo-item value">
 剧情片
 </dd>, <dd class="basicInfo-item value">
 中影集团等
 </dd>, <dd class="basicInfo-item value">
 中国
 </dd>, <dd class="basicInfo-item value">
 博纳影业集团、中国电影集团公司
 </dd>, <dd class="basicInfo-item value">
 韩三平、黄建新
 </dd>, <dd class="basicInfo-item value">
 王兴东、陈宝光
 </dd>, <dd class="basicInfo-item value">
 韩三平、黄建新（总制片人）<sup class="sup--normal" data-ctrmap=":3," data-sup="3">
 [3]</sup><a class="sup-anchor" name="ref_[3]_5580889"> </a>
 </dd>, <dd class="basicInfo-item value">
 唐国强、张国立、许晴、刘劲、陈坤、王伍福
 </dd>, <dd class="basicInfo-item value">
 135 分钟
 </dd>, <dd class="basicInfo-item value">
 2009年9月16日（中国内地）
 </dd>, <dd class="basicInfo-item value">
 43000 万元
 </dd>, <dd class="basicInfo-item value">
 汉语普通话、英语等
 </dd>, <dd class="basicInfo-item value">
 彩色
 </dd>, <dd class="basicInfo-item value">
 2009年
 </dd>, <dd class="basicInfo-item valu

In [26]:
for a in film_dir_array:
    print(a[0])

建国大业
十月围城
赤壁(下)
三枪拍案惊奇
风声
南京！南京！
游龙戏凤
疯狂的赛车
大内密探灵灵狗
非常完美
窃听风云
喜羊羊与灰太狼之牛气冲天
花木兰
麦兜响当当
刺陵
风云2
机器侠
气喘吁吁
家有喜事2009
东邪西毒:终极版
铁人
海角七号
白银帝国
爱得起
倔强萝卜
窈窕绅士
高兴
金钱帝国
熊猫大侠
高考1977
寻找成龙
扑克王
追影
午夜出租车
夜·店
斗牛
狼灾记
沂蒙六姐妹
麦田
熊猫回家路
马兰花
火星没事
恋爱前规则
红河
惊天动地
爱有来生
完美新娘
机动部队-同袍
秋喜
超能少年之烈维塔任务
齐天大圣前传
神兵小将
第一军规
隋朝来客
乐火男孩
袁崇焕
保持爱你
亲密
战无双
我的唐朝兄弟
麋鹿王
回路
矮仔多情
七天爱上你
欠我十万零五千
夜玫瑰
精灵女孩小卓玛
男生贾里新传
二十四城记
天下第一串
跳出去
爱情故事
盗版猫
老五的奥斯卡
袁隆平
喊过岭的故事
车票
寻找尘微
羊肉泡馍麻辣烫
走着瞧
寻找智美更登
透析
谁动了我的幸福
苹果树下的爱情
不能没有你
心魔
意外
唐山大地震
让子弹飞
非诚勿扰2
狄仁杰之通天帝国
叶问2,宗师传奇
赵氏孤儿
大兵小将
大笑江湖
山楂树之恋
锦衣卫
精武风云·陈真
越光宝盒
全城热恋
杜拉拉升职记
枪王之王
喜羊羊与灰太狼2
孔子
全城戒备
西风烈
剑雨
花田囍事2010
未来警察
唐伯虎点秋香2
恋爱通告
线人(2010)
苏乞儿(2010)
决战刹马镇
七十二家租客
美丽密令
魔侠传之唐吉可德?
人在囧途
午夜心跳
东风雨
嘻游记
龙凤店
七小罗汉
密室之不可告人
荒村公寓
我的野蛮女友2
长江七号爱地球
异度公寓
无人驾驶
火龙对决
绝命岛
完美嫁衣
月满轩辕诗
第一书记
岁月神偷
80’后
海洋天堂
黑猫警长
李小龙,我的兄弟
叶问前传
爱出色
我的美女老板
翡翠明珠
新康定情歌
举起手来2
三笑之才子佳人
等着你回来
出水芙蓉
爱情维修站
大玩家
功夫咏春
超蛙战士之初露锋芒
跟我的前妻谈恋爱
一路有你
爱情36计
虎王归来
庐山恋2010
摇摆de婚约
活该你单身
恋人絮语(2010)
铠甲勇士之帝皇侠
前度
绑架冰激凌
终极匹配
感情生活
志明与春娇
大劫难
虹猫蓝兔火凤凰
打擂台
日照重庆
异空危情
梦回金沙城
如梦
我们天上见
精舞门2
西游新传
马文的战争
北纬31度录

同城邂逅
生死96小时
老腔
致命怪谈
山花烂漫时
玩命剧组
判我有罪
四渡赤水
爱在深秋
午夜寻访录
隔壁惊魂
恋爱教父之三个坏家伙
爸爸的木房子
难忘金银滩
勿忘初心
假装看不见之电影大师
东宫皇子
绝不饶恕
你是不是我的爱人
爱上处女座
T台魔王
猿创世界之熊孩子部落
丢羊
飞天窑女
先锋之那时青春
那美
脱单宝典
锅是铁
不朽的时光
皇帝的新装之圣诞大明星
诡咒
返乡
回到原点（重映）
小小校长
一夜危机
谎言大爆炸
最后证言
寻找心中的你
爱情不等式
风口青春
被劫持的爱情
回乡逗儿
一棵心中的许愿树
军号密码
何去何从
非同小可
不是钱的事
岛囧
看见我和你
流金
小角色
夺命推理
疯狂的疯狂
布基兰
情比山高
天使请吻我
心语阳光
莫斯科离大同不远
太阳河
我很帅
人在驴途
非常绑架
红颜容
断片儿
骆驼客之箭在弦上
求生者
低碳爱情
在雨中
爱的钟声
钢板咏叹调
水滴之梦
自我救赎
甜水谣
谁是儿子
兄弟之北漂歌手
燕振昌
斗艳
恋
哑孩儿
皮绳上的魂
蹴鞠
爱的蟹逅
大山里的春天
我的女神女汉子
战狼2
羞羞的铁拳
功夫瑜伽
西游伏妖篇
芳华
乘风破浪
大闹天竺
悟空传
追龙
三生三世十里桃花
妖猫传
杀破狼贪狼
缝纫机乐队
建军大业
拆弹专家
嫌疑人X的献身
妖铃铃
空天猎
机器之血
心理罪
奇门遁甲
记忆大师
二代妖精之今生有幸
绣春刀·修罗战场
心理罪之城市之光
解忧杂货店
京城81号2
喜欢你
逆时营救
二十二
非凡任务
十万个冷笑话2
大卫贝肯之倒霉特工熊
父子雄兵
傲娇与偏见
决战食神
鲛珠传
赛尔号大电影6：圣者无敌
十八洞村
东北往事之破马张飞
追捕
七十七天
游戏规则
绝世高手
冈仁波齐
绑架者
大护法
麦兜响当当2017
密战
原谅他77次
血战湘江
反转人生
狂兽
欢乐喜剧人
龙之战
荡寇风云
闪光少女
破·局
疯岳撬佳人
明月几时有
合约男女
巨额来电
这就是命
昆塔：反转星球
引爆者
常在你左右
猪猪侠之英雄猪少年
大耳朵图图之美食狂想曲
李雷和韩梅梅
钢铁飞龙之再见奥特曼
有完没完
黑白迷宫
麻烦家族
玩偶奇兵
青禾男高
阿唐奇遇
烽火芳菲
“吃吃”的爱
暴雪将至
三只小猪2
二次初恋
忠爱无言
血狼犬
天生不对
嘉年华
邹碧华
相爱相亲
相声大电影之我要幸福
美好的意外
缉枪
抢红
健忘村
豆福传
神秘家族
提着心吊着

In [33]:
start = time.time()

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.96 Safari/537.36'

headers={'User-Agent':user_agent}

film_dir_act = {}

for info in film_dir_array:
    
    search = info[0]
    
    film_dir_act[search] = {}
    
    try:
        
        url = f'http://baike.baidu.com/item/{search}'

        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print('error! fail to retrieve')
            #break

        response.encoding = response.apparent_encoding

        html = response.text

        soup = bs4.BeautifulSoup(html, 'html.parser')

        seq1 = 'None'

        seq2 = 'None'

        link1 = soup.find_all('dt', class_='basicInfo-item name')

        if len(link1) == 0:

            meta = soup.find_all('div', class_='para')

            index = 0

            for n, v in enumerate(meta):

                if '演员' in v or '制片' in v or '导演' in v or '影视' in v:

                    index = n

            code = meta[index].find('a').get('data-lemmaid', None)

            url = f'http://baike.baidu.com/item/{search}/{code}'

            response = requests.get(url, headers=headers)

            if response.status_code != 200:
                print('error! fail to retrieve')
            #break

            response.encoding = response.apparent_encoding

            html = response.text

            soup1 = bs4.BeautifulSoup(html, 'html.parser')

            link3 = soup1.find_all('dt', class_='basicInfo-item name')

            for n, v in enumerate(link3):

                info = "".join(v.text.split())

                if '导演' in info:

                    seq1 = n

                if '主演' in info or '演员' in info:

                    seq2 = n

            link4 = soup1.find_all('dd', class_='basicInfo-item value')

            try: 

                dir_list = link4[seq1].find_all('a')

                if len(dir_list) > 1:

                    directors = [stuff.text.strip('\n') for stuff in dir_list]

                    director = "".join(str(item) + ', ' for item in directors)

                else:

                    director = link4[seq1].find('a').text

            except:

                director = link4[seq1].text

            try:

                act_list = link4[seq2].find_all('a')

                if len(act_list) > 1:

                    actors = [stuff.text.strip('\n') for stuff in act_list]

                    actor = "".join(str(item) + ', ' for item in actors)


                else:

                    actor = link4[seq2].find('a').text

            except:

                actor = 'None'
                
                        
            film_dir_act[search]['dir'] = director
    
            film_dir_act[search]['act'] = actor

            print(f"{search} is directed by {director} and casted {actor}")

        else:

            for n, v in enumerate(link1):

                info = "".join(v.text.split())

                if '导演' in info:

                    seq1 = n

                if '主演' in info or '演员' in info:

                    seq2 = n

            link2 = soup.find_all('dd', class_='basicInfo-item value')

            try: 

                dir_list = link2[seq1].find_all('a')

                if len(dir_list) > 1:

                    directors = [stuff.text.strip('\n') for stuff in dir_list]

                    director = "".join(str(item) + ', ' for item in directors)

                else:

                    director = link2[seq1].find('a').text

            except:

                director = link2[seq1].text

            try:

                act_list = link2[seq2].find_all('a')

                if len(act_list) > 1:

                    actors = [stuff.text.strip('\n') for stuff in act_list]

                    actor = "".join(str(item) + ', ' for item in actors)


                else:

                    actor = link2[seq2].find('a').text

            except:

                actor = 'None'
                
            film_dir_act[search]['dir'] = director
    
            film_dir_act[search]['acta'] = actor

            print(f"{search} is directed by {director} and casted {actor}")
            
    except:
        
        print(f'we failed to extract info of {info}')

    
    time.sleep(8)

end = time.time()

print(str(end - start))

# now that retrieved all this data from the NYT, how can we easily write it to a file?
# let us use pickle to do that in 2-3 easy steps

import pickle

# 1st: open a file in "write binary" (wb) mode
datafile = open("web_scraping.pickled", "wb")

# 2nd: call pickle.dump(). First argument: the data structure or variable to be saved to the file
# Second argument: the file handle (or "file reference") opened above in "wb" mode

pickle.dump(film_dir_act, datafile)

# 3rd: close that file to make sure everything was written to disk!
datafile.close()


建国大业 is directed by 
韩三平、黄建新
 and casted None
十月围城 is directed by 
鞠觉亮,邹集城
 and casted None
we failed to extract info of 开本
三枪拍案惊奇 is directed by 
张艺谋
 and casted None
风声 is directed by 
高群书、陈国富
 and casted None
南京！南京！ is directed by 陆川 and casted 刘烨, 高圆圆, 范伟, 秦岚, 中泉英雄, 
游龙戏凤 is directed by 
刘伟强
 and casted 刘德华, 舒淇, 张涵予, 
疯狂的赛车 is directed by 
宁浩
 and casted 黄渤, 戎祥, 
大内密探灵灵狗 is directed by 王晶 and casted 古天乐, 徐熙媛, 佟大为, 小宋佳, 樊少皇, 吴君如, 
非常完美 is directed by 胡庶 and casted None
窃听风云 is directed by 崔东勋 and casted None
喜羊羊与灰太狼之牛气冲天 is directed by 
赵崇邦
 and casted 阿牛, 毛方圆, 巫迪文, 祖丽晴, 邓玉婷, 梁颖, 
we failed to extract info of 所处时期
麦兜响当当 is directed by 
谢立文
 and casted 吴君如, 黄秋生, 詹瑞文, 
刺陵 is directed by 朱延平 and casted 周杰伦, 林志玲, 陈道明, 
风云2 is directed by 
彭顺、彭发
 and casted 郭富城, 郑伊健, 任达华, 谢霆锋, 蔡卓妍, 唐嫣, 何家劲, 林雪, 
机器侠 is directed by 刘镇伟 and casted 孙俪, 方力申, 胡军, 郑中基, 甘薇, 
气喘吁吁 is directed by 郑重 and casted 葛优, 约翰·萨维奇, 林熙蕾, 刘桦, 张秋芳, 陈柏霖, 梁天,  , 吴晴,  , 李菁,  , 于莎莎,  , 纪连海,  , 郝凯瑞,  , 
家有喜事2009 is directed by 
谷德

爱情36计 is directed by 
何美义
 and casted None
虎王归来 is directed by 黄军 and casted None
庐山恋2010 is directed by 
张瑜
 and casted 秦岚, 李晨, 方中信, 张瑜, 
摇摆de婚约 is directed by 
鄢颇，肖蔚鸿
 and casted 姚晨, 朱雨辰, 郭晓冬, 
活该你单身 is directed by 蔡心 and casted 林心如, 贺军翔, 爱戴, 吴大维, 陶虹, 刘芸, 
we failed to extract info of ('恋人絮语(2010)', ['曾国祥', '尹志文'])
铠甲勇士之帝皇侠 is directed by 
郑国伟
 and casted 唐禹哲, 郭帅, 袁成杰, 陈真希, 杨亚, 廖晓晨, 董文军, 吴建飞, 
we failed to extract info of 改编自
绑架冰激凌 is directed by 
古宇
 and casted 杜汶泽, 孙兴, 巫刚, 赵美彤, 瞿颖, 叶一茜, 李菲儿, 苏青, 
终极匹配 is directed by 
程珑
 and casted 吴佩慈, 孟广美, 顾莉雅, 徐啸力, 连晋, 
感情生活 is directed by 
杨竞泽
 and casted None
志明与春娇 is directed by 彭浩翔 and casted 杨千嬅, 余文乐, 谷祖琳, 陈逸宁, 方皓玟, 
大劫难 is directed by 
肖风
 and casted None
虹猫蓝兔火凤凰 is directed by 贺梦凡 and casted 何炅, 谢娜, 杜海涛, 李维嘉, 吴昕, 袁慧敏, 王娜, 刘娟, 
we failed to extract info of 擂台
日照重庆 is directed by 王小帅 and casted 王学圻, 秦昊, 子义, 范冰冰, 
异空危情 is directed by 张海靖 and casted 冯德伦, 应采儿, 唐一菲, 蒋雅文, 陈键锋, 张檬, 
梦回金沙城 is directed by 陈德明 and casted None
如梦 is directed by 罗卓瑶 a

钢琴木马 is directed by 
崔斯韦
 and casted None
竞雄女侠·秋瑾 is directed by 邱礼涛 and casted 黄奕, 黄秋生, 郑嘉颖, 杜宇航, 夏文汐, 张颂文, 
惊沙 is directed by 
安战军
 and casted None
we failed to extract info of 玩家人数
非常营救 is directed by 毛坚 and casted 赵文瑄, 胡影怡, 任帅, 董彦麟, 
你是哪里人 is directed by 蒋旭 and casted 杨恭如, 苗皓钧, 李斯羽, 唐文龙, 潘阳, 姚笛, 张俪, 林菀, 
幸存日 is directed by 
道格拉斯·阿诺科斯基
 and casted 肖恩·阿什莫
一路惊心 is directed by 
李伟
 and casted None
星海 is directed by 
李前宽、肖桂云
 and casted 苏有朋, 徐若瑄, 苏嘉航, 王成阳, 
we failed to extract info of ISBN
云下的日子 is directed by 闫然 and casted 张涵予, 于娜, 
大太阳 is directed by 
杨亚洲
 and casted 倪萍, 刘佩琦, 蒋勤勤, 常戎, 
琴动我心 is directed by 
周伟
 and casted 高曙光, 邬君梅, 
无界之地 is directed by 
徐子牧
 and casted 钟镇涛, 甘薇, 林雪,  , 谭耀文,  , 熊乃瑾,  , 郑浩南,  , 谢君豪,  , 李彩桦,  , 周彦岑,  , 
we failed to extract info of ('正,青春', ['王立新', '胡韵'])
完美的爱 is directed by 王立新 and casted 徐啸力, 杨杏, 王胜利,  , 马志安,  , 王立新,  ,  , 
we failed to extract info of 歌曲语言
百年情书 is directed by 金舸 and casted 王柏杰, 蒋梦婕, 李雪健, 
we failed to extract info of 出版时间
郎在对门唱山歌 is di

恋爱三万英尺 is directed by 
杨子
 and casted 方力申, 谢君豪, 刘羽琦, 冯丹滢, 郑晓宁, 
热爱岛 is directed by 甘国亮 and casted 任达华, 张震, 吴镇宇, 宋佳, 陈冲, 文咏珊, 
we failed to extract info of ('乌龙戏凤（2012）', ['罗安得', 'nan'])
饮食男女2012 is directed by 
曹瑞原
 and casted 蓝正龙
地域无门 is directed by 
吴耀权
 and casted 立威廉, 陈紫函, 谭耀文, 柳海龙, 李兆基, 魏璐, 
死亡迷局 is directed by 
鲁坚
 and casted None
早见，晚爱 is directed by 
刘国昌
 and casted 周渝民, 童瑶, 曹云金, 白羽, 叶倩云, 
一起飞 is directed by 
张蠡
 and casted None
we failed to extract info of ('玩命,时光', ['曲立男', 'nan'])
下一个奇迹 is directed by 梁修身 and casted 谢霆锋, 张卫健, 吴秀波, 张芷溪, 海清, 王鸥, 李光洁, 林永健, 王智, 丁子峻, 韩雨芹, 
结婚狂想曲 is directed by 
杨郎从
 and casted 李承铉, 安以轩, 黄一飞, 洪剑涛, 
犀利人妻 is directed by 徐辅军, 曾丽珍,  and casted 隋棠, 温升豪, 王宥胜, 朱芯仪, 
we failed to extract info of 外文名
万箭穿心 is directed by 
王竞
 and casted None
请叫我英雄 is directed by 
傅咏
 and casted 吴镇宇, 乔任梁, 林雪, 蓝燕, 关颖, 钟丽缇, 
台北飘雪 is directed by 
霍建起
 and casted 陈柏霖, 童瑶, 杨佑宁, 莫子仪, 
非常绑匪 is directed by 李学兵 and casted 张默, 邬祯琳, 姜武, 张光北, 艾俊龙, 
狂奔蚂蚁 is directed by 
潘昊、葛文喆
 and casted

青春派 is directed by 刘杰 and casted 董子健, 秦海璐, 安悦溪, 
太极侠 is directed by 
基努·里维斯
 and casted 陈虎, 基努·李维斯, 莫文蔚, 于海, 叶青, 任达华, 
魁拔之大战元泱界 is directed by 
王川
 and casted None
81号农场之保卫麦咭 is directed by 徐克 and casted 范楚绒, 李晔, 王衡, 
意外的恋爱时光 is directed by 
黎志
 and casted 房祖名, 郭采洁, 李菲儿, 于莎莎, 
枕边有张脸 is directed by 
赵小溪
 and casted 吴其江, 赵仕瑾, 王翀, 赵叶索, 刘妙, 
圣诞玫瑰 is directed by 
杨采妮
 and casted None
一场风花雪月的事 is directed by 高群书 and casted 杨颖, 黄晓明, 在熙, 巍子, 许峻豪, 
同谋 is directed by 
彭顺
 and casted None
we failed to extract info of ('电梯惊魂(2013)', ['宁敬武', 'nan'])
5月天诺亚方舟 is directed by 
刘名峰、五月天
 and casted None
夺宝联盟 is directed by 崔东勋 and casted 金允石, 金惠秀, 李政宰, 全智贤, 金秀贤, 金海淑, 任达华, 吴达庶, 李心洁, 曾国祥, 
诡婴吉咪 is directed by 梁普智 and casted 林峯, 盛君, 
we failed to extract info of 制作单位
孤岛惊魂2 is directed by 
钟继昌
 and casted 邓家佳, 陈志朋, 
一路顺疯 is directed by 
叶伟民
 and casted 陈思诚, 包贝尔, 蒋梦婕, 鲍起静, 刘桦, 李欣汝, 黄小蕾, 
201314 is directed by 
孔令晨
 and casted None
制服 is directed by 
王光利
 and casted None
金刚王 is directed by 罗永昌 and casted 释延能, 

...

炸裂青春 is directed by 
牟天翔
 and casted  , 巴多, 
斗魂之熊孩子 is directed by 
纪磊
 and casted 纪凯童, 魏辰妃, 申泰明, 
麻辣学院 is directed by 
李金瀚
 and casted 吕孔维, 黄品源, 蒋欣, 董立范, 唐嘉悦, 唐艺心, 李炳辰, 关燕萍, 郭美美, 
we failed to extract info of 代表作
蛋计划 is directed by 李扬武 and casted None
夜闯寡妇村 is directed by 
邢博
 and casted 于朦胧,  , 
we failed to extract info of 在线播放平台
蜜月计划 is directed by 
颢然
 and casted 陈燃, 汪洋, 齐超, 
绝技情缘之艺魂 is directed by 
张建成
 and casted 杜奕衡,  , 童苡萱,  , 曹帅,  , 
识色，幸也 is directed by 刘欣翰 and casted 范逸臣, 黄奕, 沈梦辰, 韩承羽, 刘之冰,  , 唐熙,  , 白纬玲,  , 万子晴,  , 
南哥 is directed by 
郑华、潘钧、王俊翔
 and casted 茹萍, 艾丽娅, 赵中华, 蒙伟明, 
我心雀跃 is directed by 
刘紫微
 and casted None
让爱活下去 is directed by   and casted 夏德俊, 郑佩佩, 周诗璇, 
灿烂这一刻 is directed by 陈果 and casted 林雪, 郝蕾, 
娘子军传奇 is directed by 孟奇,  ,  and casted 刘一含, 战卫华,  , 邓杨宁,  , 孙菲菲,  ,  , 
钟馗传奇之岁寒三友 is directed by 
晓寒
 and casted None
逗逗迪迪之美梦年年 is directed by 
王龙波，陆锦明
 and casted None
纹身师 is directed by Peter Burger and casted None
童话先生 is directed by 
李宗泰
 and casted None
来自月亮的我 i

战神纪 is directed by 哈斯朝鲁 and casted 陈伟霆, 林允, 胡军, 倪大红, 李光洁, 赵立新, 张歆艺, 巴森扎布, 涂们, 巴音, 咏梅, 
we failed to extract info of 游戏语言
妈妈咪鸭 is directed by 
赵锐、克里斯·詹金斯
 and casted None
we failed to extract info of 类别
脱单告急 is directed by 
柯孟融
 and casted None
低压槽：欲望之城 is directed by 
张家辉
 and casted None
冰封侠:时空行者 is directed by 
叶伟民
 and casted None
奇葩朵朵 is directed by 
李欣、李洋
 and casted None
恐龙王 is directed by   and casted 吕佩玉
we failed to extract info of 分布区域
大路朝天 is directed by 
李伟
 and casted None
we failed to extract info of 制作人
坏爸爸 is directed by 
张跃龄
 and casted None
猫与桃花源 is directed by 
王微
 and casted None
we failed to extract info of 对白语言
藏北秘岭-重返无人区 is directed by 
饶子君
 and casted None
中国合伙人2 is directed by 
刘亚当
 and casted None
泡芙小姐 is directed by 张歆艺 and casted 张歆艺, 王栎鑫, 谭维维, 陈哲远, 于莎莎, 曾一竣, 孙骁骁, 包贝尔, 蒋欣, 张静初, 张天爱, 秦岚, 江珊, 
天渠 is directed by 柏麟 and casted None
功夫联盟 is directed by 
刘镇伟
 and casted None
中国蓝盔 is directed by 
尚昌仪
 and casted None
我说的都是真的 is directed by 
刘仪伟
 and casted None
we 

他是一只狗 is directed by 
孙健、杨昕雨
 and casted None
天梦 is directed by 
许文琦
 and casted None
只想好好和你在一起 is directed by 
屈珂羽
 and casted 刘思伟,  ,  ,  ,  ,  , 
奔 is directed by 
黄中平
 and casted None
we failed to extract info of 连载网站
雾都神探之绝情箭 is directed by 孙铎 and casted 孙骁潇, 余薇薇, 李日詹, 孙丹丹, 许可, 程爽, 李耕, 张春年, 
醉佳导演 is directed by 武圣基 and casted 贺军翔, 江若琳, 刘宇,  , 姚橹,  , 梁天,  ,  , 
那条河 is directed by 张锐 and casted 张杨果而,  ,  ,  ,  ,  , 
堡藏 is directed by 刘峰 and casted 甘林熠, 安赛龙, 王禹铮, 
找爱之旅 is directed by   and casted  ,  ,  , 
we failed to extract info of ('我想和你们在一起', ['颜芳', 'nan'])
we failed to extract info of 小说类型
二麻租媳妇 is directed by   and casted  ,  , 
渠首欢歌 is directed by 尹海峰 and casted 丁锐, 尚国伟, 郝光, 惠娟艳, 王艺禅, 赵筱青, 
we failed to extract info of ISBN
星溪的三次奇遇 is directed by 
竹原青
 and casted None
女他 is directed by 周圣崴 and casted None
we failed to extract info of 最新版本
we failed to extract info of 纸张
半个月亮 is directed by 
毛卫宁
 and casted None
we failed to extract info of 开本
we failed to extract info of 出版时间

武陵山上的星光 is directed by 
张塞君
 and casted None
共同命运 is directed by 曲江涛, 邓斐, 刘畅, 王强,  and casted 马云, 姚明, 
当我们海阔天空 is directed by   and casted 刘秋实, 刘奕畅, 何琢言,  ,  , 
少女宿舍 is directed by 
杨盼盼、曹尚水
 and casted None
大大哒 is directed by 
周青元
 and casted None
花椒之味 is directed by 
麦曦茵
 and casted None
we failed to extract info of 注音
we failed to extract info of 连载网站
一切如你 is directed by 
黄宏、黄兆函、傅绍杰、张楠、李譞
 and casted 于蓝
we failed to extract info of 连载网站
蓝色生死恋 is directed by 
尹锡湖
 and casted 宋承宪, 宋慧乔, 元斌, 韩彩英, 韩娜娜, 
妈妈，我想你 is directed by 
康锐
 and casted None
50米之恋 is directed by 
李依理
 and casted None
幸福的小满 is directed by 
钟海
 and casted None
好小子，好功夫 is directed by 
崔喆、胡方易、李木子
 and casted None
we failed to extract info of 代表作品
we failed to extract info of 在线播放平台
六欲天 is directed by 
祖峰
 and casted None
we failed to extract info of 语法
we failed to extract info of 演唱者
we failed to extract info of ('猫公主苏菲', ['潘智超', '郭攀'])
学区房72小时 is directed by 陈晓鸣 and casted 管轩, 徐幸, 傅淼, 
we failed to extract info of ('那一夜

we failed to extract info of 主演
禁锢之地 is directed by 
吴奇
 and casted None
we failed to extract info of 连载网站
we failed to extract info of 释义
大陈岛誓言 is directed by 
陈华杰
 and casted None
23601.557836294174


In [2]:
data = pd.read_pickle('web_scraping.pickled')

In [38]:
data

{'建国大业': {'dir': '\n韩三平、黄建新\n', 'acta': 'None'},
 '十月围城': {'dir': '\n鞠觉亮,邹集城\n', 'acta': 'None'},
 '赤壁(下)': {},
 '三枪拍案惊奇': {'dir': '\n张艺谋\n', 'acta': 'None'},
 '风声': {'dir': '\n高群书、陈国富\n', 'acta': 'None'},
 '南京！南京！': {'dir': '陆川', 'act': '刘烨, 高圆圆, 范伟, 秦岚, 中泉英雄, '},
 '游龙戏凤': {'dir': '\n刘伟强\n', 'acta': '刘德华, 舒淇, 张涵予, '},
 '疯狂的赛车': {'dir': '\n宁浩\n', 'acta': '黄渤, 戎祥, '},
 '大内密探灵灵狗': {'dir': '王晶', 'acta': '古天乐, 徐熙媛, 佟大为, 小宋佳, 樊少皇, 吴君如, '},
 '非常完美': {'dir': '胡庶', 'acta': 'None'},
 '窃听风云': {'dir': '崔东勋', 'acta': 'None'},
 '喜羊羊与灰太狼之牛气冲天': {'dir': '\n赵崇邦\n', 'acta': '阿牛, 毛方圆, 巫迪文, 祖丽晴, 邓玉婷, 梁颖, '},
 '花木兰': {},
 '麦兜响当当': {'dir': '\n谢立文\n', 'act': '吴君如, 黄秋生, 詹瑞文, '},
 '刺陵': {'dir': '朱延平', 'acta': '周杰伦, 林志玲, 陈道明, '},
 '风云2': {'dir': '\n彭顺、彭发\n', 'acta': '郭富城, 郑伊健, 任达华, 谢霆锋, 蔡卓妍, 唐嫣, 何家劲, 林雪, '},
 '机器侠': {'dir': '刘镇伟', 'acta': '孙俪, 方力申, 胡军, 郑中基, 甘薇, '},
 '气喘吁吁': {'dir': '郑重',
  'acta': '葛优, 约翰·萨维奇, 林熙蕾, 刘桦, 张秋芳, 陈柏霖, 梁天, \xa0, 吴晴, \xa0, 李菁, \xa0, 于莎莎, \xa0, 纪连海, \xa0, 郝凯瑞, \xa0, '},
 '家有喜事2009': {'

In [39]:
df2 = pd.DataFrame(data).T.reset_index()

In [41]:
df2['dir'] = df2['dir'].astype('str')

In [42]:
df2['dir'] = [Dir.strip('\n').replace('xa0','').replace('、',',') for Dir in df2['dir']]

In [43]:
df2

,index,dir,acta,act
0,建国大业,"韩三平,黄建新",None,NaN
1,十月围城,"鞠觉亮,邹集城",None,NaN
2,赤壁(下),nan,NaN,NaN
3,三枪拍案惊奇,张艺谋,None,NaN
4,风声,"高群书,陈国富",None,NaN
...,...,...,...,...
2650,风雨同仁堂,nan,NaN,NaN
2651,禁锢之地,吴奇,None,NaN
2652,相亲时代,nan,NaN,NaN
2653,薪火相传,nan,NaN,NaN


In [45]:
df_merged = pd.merge(df, df2, left_on='name', right_on='index', how = 'left')

### problems in current web_scraping
1. 电影与电视剧有问题
2. 电影与真人秀冲突 
3. 与动画冲突 神兵小将
4. 和开幕歌曲冲突

In [54]:
df.name.value_counts()

海门深处          2
旺扎的雨靴         2
特别追踪          2
五月天人生无限公司     1
爱情银行          1
             ..
刺陵            1
石器时代之百万大侦探    1
死亡之谜之双鱼玉佩     1
山花烂漫时         1
攀登者           1
Name: name, Length: 2655, dtype: int64

In [56]:
df_merged['dir_list'] = [item[1] for item in df_merged['dict']]

In [58]:
df_merged

,name,box2019,director1,director2,ms1,fs1,ms2,fs2,guests,dir_x,dict,index,dir_y,acta,act,dir_list
0,建国大业,5.369108,韩三平,黄建新,唐国强,许晴,张国立,邬君梅,陈坤,"韩三平,黄建新","(建国大业, [韩三平, 黄建新])",建国大业,"韩三平,黄建新",None,NaN,"[韩三平, 黄建新]"
1,十月围城,3.755794,陈德森,nan,甄子丹,范冰冰,梁家辉,李宇春,谢霆锋,"陈德森,nan","(十月围城, [陈德森, nan])",十月围城,"鞠觉亮,邹集城",None,NaN,"[陈德森, nan]"
2,赤壁(下),3.368599,吴宇森,nan,梁朝伟,赵薇,金城武,林志玲,佟大为,"吴宇森,nan","(赤壁(下), [吴宇森, nan])",赤壁(下),nan,NaN,NaN,"[吴宇森, nan]"
3,三枪拍案惊奇,3.329879,张艺谋,nan,孙红雷,闫妮,小沈阳,nan,赵本山,"张艺谋,nan","(三枪拍案惊奇, [张艺谋, nan])",三枪拍案惊奇,张艺谋,None,NaN,"[张艺谋, nan]"
4,风声,2.891058,高群书,陈国富,黄晓明,周迅,苏有朋,李冰冰,张涵予,"高群书,陈国富","(风声, [高群书, 陈国富])",风声,"高群书,陈国富",None,NaN,"[高群书, 陈国富]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2653,风雨同仁堂,0.000109,马崇杰,nan,张永和,蓝荫海,王新纪,nan,nan,"马崇杰,nan","(风雨同仁堂, [马崇杰, nan])",风雨同仁堂,nan,NaN,NaN,"[马崇杰, nan]"
2654,禁锢之地,0.000109,吴奇,nan,周柏豪,王真儿,罗伯特·克耐普,张丹妮,nan,"吴奇,nan","(禁锢之地, [吴奇, nan])",禁锢之地,吴奇,None,NaN,"[吴奇, nan]"
2655,相亲时代,0.000099,蒋君君,nan,郭烁杰,李欣烨,宁小花,廖娟,nan,"蒋君君,nan","(相亲时代, [蒋君君, nan])",相亲时代,nan,NaN,NaN,"[蒋君君, nan]"
2656,薪火相传,0.000099,王鼎,nan,刘劲,nan,刘锦航,nan,nan,"王鼎,nan","(薪火相传, [王鼎, nan])",薪火相传,nan,NaN,NaN,"[王鼎, nan]"


In [59]:
df_merged['ind'] = [1 if a.split(',')[0] in b else 0for a, b in zip(df_merged['dir_y'], df_merged['dir_list'])]

In [83]:
df_merged[df_merged['name'] == '追影']['dir_y']

32    吴镇宇，麦子善
Name: dir_y, dtype: object

In [64]:
wrong_info = df_merged[df_merged['ind'] == 0].reset_index(drop=True)

In [66]:
wrong_info.to_csv('wrong_info.csv')

In [1]:
%run ~/a_Research/Input/Geographic_Data.py
